In [3]:
import requests
import pandas as pd
import gmaps
import json
import csv
from keys import gapikey

gmaps.configure(api_key=gapikey)

with open('zcta/zcta.json') as f:
    geoj = json.load(f)

with open('resources/zip_targets.csv', newline='\n', encoding='utf-8-sig') as e:
    zipdf = pd.read_csv(e)
    ziplist = list(zipdf['zip'])
    
features = geoj['features']
feature_col = []

for feature in features:
    try:
        feature_geoid = int(feature['properties']['GEOID10'])
        if feature_geoid in ziplist:
            feature_col.append(feature)
            
    except ValueError:
            print(f'Value Error {b}')
                  
geoj_scrubbed = {'type': 'FeatureCollection', 'features': feature_col}

In [4]:
zcta_layer = gmaps.geojson_layer(geoj_scrubbed)
fig = gmaps.figure()
fig.add_layer(zcta_layer)
fig

Figure(layout=FigureLayout(height='420px'))